# CNN_3channel_moredata

Abstract:
- single channel: band_avg
- CNN, small net

Result:
- Kaggle score: 

References:
- https://www.kaggle.com/ivalmian/simple-svd-xgboost-baseline-lb-35
- https://www.kaggle.com/arieltci/a-keras-prototype-0-21174-on-pl

## 1. Preprocess

### Import pkgs

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import log_loss, accuracy_score
from IPython.display import display

%matplotlib inline

In [2]:
import os
import time
import zipfile
import lzma
import pickle
from PIL import Image
from shutil import copy2
import gc

import cv2

### Run name

In [3]:
project_name = 'SC_Iceberg_Classifier'
step_name = 'CNN_3channel_moredata'
date_str = time.strftime("%Y%m%d", time.localtime())
time_str = time.strftime("%Y%m%d_%H%M%S", time.localtime())
run_name = project_name + '_' + step_name + '_' + time_str
print('run_name: ' + run_name)
t0 = time.time()

run_name: SC_Iceberg_Classifier_CNN_3channel_moredata_20180108_203934


### Basic folders

In [4]:
cwd = os.getcwd()
input_path = os.path.join(cwd, 'input')
log_path = os.path.join(cwd, 'log')
model_path = os.path.join(cwd, 'model')
output_path = os.path.join(cwd, 'output')
print('input_path: ' + input_path)
print('log_path: ' + log_path)
print('model_path: ' + model_path)
print('output_path: ' + output_path)

input_path: D:\Kaggle\statoil-iceberg-classifier-challenge\input
log_path: D:\Kaggle\statoil-iceberg-classifier-challenge\log
model_path: D:\Kaggle\statoil-iceberg-classifier-challenge\model
output_path: D:\Kaggle\statoil-iceberg-classifier-challenge\output


### Basic functions

In [5]:
import sys
def describe(arr):
    print(arr.shape, arr.min(), arr.max(), sys.getsizeof(arr))

### Load data

In [6]:
sample_submission_path = os.path.join(input_path, 'sample_submission.csv')
sample_submission = pd.read_csv(sample_submission_path)
print(sample_submission.shape)
sample_submission.head(2)

(8424, 2)


,id,is_iceberg
0,5941774d,0.5
1,4023181e,0.5


In [7]:
is_iceberg_path = os.path.join(input_path, 'is_iceberg.p')

y_data = pickle.load(open(is_iceberg_path, mode='rb'))

describe(y_data)

(1604,) 0 1 12936


In [8]:
%%time
#Load orignal data
inc_angle_data_path = os.path.join(input_path, 'inc_angle_data.p')
inc_angle_test_path = os.path.join(input_path, 'inc_angle_test.p')

inc_angle_data = pickle.load(open(inc_angle_data_path, mode='rb'))
inc_angle_test = pickle.load(open(inc_angle_test_path, mode='rb'))

print(inc_angle_data.shape)
print(inc_angle_test.shape)

(1604,)
(8424,)
Wall time: 2.5 ms


In [9]:
%%time
#Load orignal data
band1_data_path = os.path.join(input_path, 'band1_data_gray.p')
band2_data_path = os.path.join(input_path, 'band2_data_gray.p')
band1_test_path = os.path.join(input_path, 'band1_test_gray.p')
band2_test_path = os.path.join(input_path, 'band2_test_gray.p')

band1_data = pickle.load(open(band1_data_path, mode='rb'))
band2_data = pickle.load(open(band2_data_path, mode='rb'))
band1_test = pickle.load(open(band1_test_path, mode='rb'))
band2_test = pickle.load(open(band2_test_path, mode='rb'))

describe(band1_data)
describe(band2_data)
describe(band1_test)
describe(band2_test)

(1604, 75, 75) 0.0 1.0 128
(1604, 75, 75) 0.0 1.0 128
(8424, 75, 75) 0.0 1.0 128
(8424, 75, 75) 0.0 1.0 128
Wall time: 1.47 s


In [10]:
%%time
#Load orignal data
band_sub_data = np.fabs(np.subtract(band1_data, band2_data))
band_sub_test = np.fabs(np.subtract(band1_test, band2_test))
band_min_data = np.minimum(band1_data, band2_data)
band_min_test = np.minimum(band1_test, band2_test)
band_max_data = np.maximum(band1_data, band2_data)
band_max_test = np.maximum(band1_test, band2_test)

describe(band_sub_data)
describe(band_sub_test)
describe(band_min_data)
describe(band_min_test)
describe(band_max_data)
describe(band_max_test)

(1604, 75, 75) 0.0 0.979400825264 72180128
(8424, 75, 75) 0.0 0.930272493331 379080128
(1604, 75, 75) 0.0 1.0 72180128
(8424, 75, 75) 0.0 1.0 379080128
(1604, 75, 75) 0.0210369138693 1.0 72180128
(8424, 75, 75) 0.0 1.0 379080128
Wall time: 1.95 s


In [11]:
%%time
x_data = np.concatenate([band_sub_data[:, :, :, np.newaxis],
                         band_min_data[:, :, :, np.newaxis],
                         band_max_data[:, :, :, np.newaxis]], axis=-1)
describe(x_data)
# del band1_data
# del band2_data
# del band_avg_data
gc.collect()

x_test = np.concatenate([band_sub_test[:, :, :, np.newaxis],
                         band_min_test[:, :, :, np.newaxis],
                         band_max_test[:, :, :, np.newaxis]], axis=-1)
describe(x_test)
# del band1_test
# del band2_test
# del band_avg_test
gc.collect()

(1604, 75, 75, 3) 0.0 1.0 216540144
(8424, 75, 75, 3) 0.0 1.0 1137240144
Wall time: 1.59 s


In [12]:
%%time
x_train, x_val, y_train, y_val = train_test_split(x_data, y_data, test_size=0.1, random_state=31)

describe(x_train)
describe(x_val)

describe(y_train)
describe(y_val)

(1443, 75, 75, 3) 0.0 1.0 194805144
(161, 75, 75, 3) 0.0 1.0 21735144
(1443,) 0 1 23112
(161,) 0 1 2600
Wall time: 171 ms


## 2. Build model

In [13]:
from keras.utils.np_utils import to_categorical # convert to one-hot-encoding
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D, GlobalMaxPooling2D, BatchNormalization, Input
from keras.layers.merge import Concatenate
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import LearningRateScheduler, TensorBoard

Using TensorFlow backend.


In [14]:
def build_model():
    model = Sequential()

    model.add(Conv2D(filters = 64, kernel_size = (3, 3), activation='relu',
                     input_shape = (x_train.shape[1], x_train.shape[2], x_train.shape[3])))
    model.add(BatchNormalization())
    model.add(Conv2D(filters = 64, kernel_size = (3, 3), activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(strides=(2,2)))
    model.add(Dropout(0.2))

    model.add(Conv2D(filters = 128, kernel_size = (3, 3), activation='relu'))
    model.add(BatchNormalization())
    model.add(Conv2D(filters = 128, kernel_size = (3, 3), activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(strides=(2,2)))
    model.add(Dropout(0.2))

    model.add(Conv2D(filters = 64, kernel_size = (3, 3), activation='relu'))
    model.add(BatchNormalization())
    model.add(Conv2D(filters = 64, kernel_size = (3, 3), activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(strides=(2,2)))
    model.add(Dropout(0.2))
    
    model.add(Flatten())
    model.add(Dense(1024, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(1024, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(units = 1, activation = 'sigmoid'))

    model.compile(optimizer = Adam(lr=1e-4), loss = 'binary_crossentropy', metrics = ['accuracy'])
    return model

In [15]:
model = build_model()
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 73, 73, 64)        1792      
_________________________________________________________________
batch_normalization_1 (Batch (None, 73, 73, 64)        256       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 71, 71, 64)        36928     
_________________________________________________________________
batch_normalization_2 (Batch (None, 71, 71, 64)        256       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 35, 35, 64)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 35, 35, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 33, 33, 128)       73856     
__________

In [16]:
def saveModel(model, run_name):
    cwd = os.getcwd()
    modelPath = os.path.join(cwd, 'model')
    if not os.path.isdir(modelPath):
        os.mkdir(modelPath)
    weigthsFile = os.path.join(modelPath, run_name + '.h5')
    model.save(weigthsFile)
# saveModel(model, 'saveModel_test')

In [17]:
def get_lr(x):
    lr = round(1e-4 * 0.995 ** x, 6)
    if lr < 5e-5:
        lr = 5e-5
    print(lr, end='  ')
    return lr

# annealer = LearningRateScheduler(lambda x: 1e-3 * 0.9 ** x)
annealer = LearningRateScheduler(get_lr)

log_dir = os.path.join(log_path, run_name)
print('log_dir:' + log_dir)
tensorBoard = TensorBoard(log_dir=log_dir)

log_dir:D:\Kaggle\statoil-iceberg-classifier-challenge\log\SC_Iceberg_Classifier_CNN_3channel_moredata_20180108_203934


In [18]:
datagen = ImageDataGenerator(
    zoom_range = 0.2,
#     horizontal_flip=True,
#     vertical_flip=True,
    rotation_range=10,
    width_shift_range=0.2,
    height_shift_range=0.2
)

In [19]:
%%time
batch_size = 32
# steps_per_epoch = 1 * len(x_train) / batch_size
hist = model.fit_generator(
    datagen.flow(x_train, y_train, batch_size=batch_size, shuffle=True),
    steps_per_epoch=100,
    epochs=300, #1 for ETA, 0 for silent
    verbose=1,
    max_queue_size=128,
    callbacks=[annealer],
    workers=32,
    validation_data=(x_val, y_val)
)

0.0001  Epoch 1/300
100/100 [==============================] - 88s 877ms/step - loss: 0.9302 - acc: 0.5568 - val_loss: 0.6936 - val_acc: 0.4534
0.0001  Epoch 2/300
100/100 [==============================] - 82s 818ms/step - loss: 0.6956 - acc: 0.6141 - val_loss: 0.7008 - val_acc: 0.4534
9.9e-05  Epoch 3/300
100/100 [==============================] - 81s 812ms/step - loss: 0.6452 - acc: 0.6353 - val_loss: 0.7029 - val_acc: 0.4534
9.9e-05  Epoch 4/300
100/100 [==============================] - 82s 818ms/step - loss: 0.6187 - acc: 0.6401 - val_loss: 0.7070 - val_acc: 0.4534
9.8e-05  Epoch 5/300
100/100 [==============================] - 82s 824ms/step - loss: 0.6233 - acc: 0.6471 - val_loss: 0.6917 - val_acc: 0.4845
9.8e-05  Epoch 6/300
100/100 [==============================] - 82s 817ms/step - loss: 0.6122 - acc: 0.6566 - val_loss: 0.6539 - val_acc: 0.6273
9.7e-05  Epoch 7/300
100/100 [==============================] - 82s 817ms/step - loss: 0.5974 - acc: 0.6508 - val_loss: 0.6115 - val

KeyboardInterrupt: 

In [20]:
# %%time
# hist = model.fit(x_train, y_train, 
#                  batch_size = 32, 
#                  verbose= 1,
# #                  callbacks = [annealer],
#                  epochs = 100, #1 for ETA, 0 for silent
#                  validation_data=(x_val, y_val))

In [21]:
final_loss, final_acc = model.evaluate(x_val, y_val, verbose=1)
print("Final loss: {0:.4f}, final accuracy: {1:.4f}".format(final_loss, final_acc))

161/161 [==============================] - 1s 8ms/step
Final loss: 0.5459, final accuracy: 0.6832


In [22]:
val_prob1 = model.predict(x_val)

# print('Val log_loss: {}'.format(log_loss(y_val, val_prob1)))
val_prob1_limit = np.clip(val_prob1, 0.00005, 0.99995)
loss = log_loss(y_val, val_prob1_limit)
print('Val log_loss: {}'.format(loss))

val_prob1_limit = np.clip(val_prob1_limit, 0.05, 0.95)
loss = log_loss(y_val, val_prob1_limit)
print('Val log_loss: {}'.format(loss))

Val log_loss: 0.5459460633812669
Val log_loss: 0.5479681968041088


In [23]:
final_acc_str = str(int(loss*10000))
run_name_acc = project_name + '_' + step_name + '_' + time_str + '_' + final_acc_str
print(run_name_acc)

SC_Iceberg_Classifier_CNN_3channel_moredata_20180108_203934_5479


In [24]:
histories = pd.DataFrame(hist.history)
histories['epoch'] = hist.epoch
print(histories.columns)
histories_file = os.path.join(model_path, run_name_acc + '.csv')
histories.to_csv(histories_file, index=False)

NameError: name 'hist' is not defined

In [ ]:
plt.plot(histories['loss'], color='b')
plt.plot(histories['val_loss'], color='r')
plt.show()
plt.plot(histories['acc'], color='b')
plt.plot(histories['val_acc'], color='r')
plt.show()

In [ ]:
saveModel(model, run_name_acc)

## 3. Predict

In [ ]:
if not os.path.exists(output_path):
    os.mkdir(output_path)
pred_file = os.path.join(output_path, run_name_acc + '.csv')
print(pred_file)

In [ ]:
test_prob = model.predict(x_test)
print(test_prob.shape)
print(test_prob[0:2])
test_prob = np.clip(test_prob, 0.05, 0.95)
print(test_prob.shape)
print(test_prob[0:2])

In [ ]:
sample_submission['is_iceberg'] = test_prob
print(sample_submission[0:2])
print(sample_submission.shape)
sample_submission.to_csv(pred_file, index=False)

In [ ]:
t1 = time.time()
print('time cost: %.2f s' % (t1-t0))
print('Done!')

In [ ]:
print(run_name_acc)